In [43]:
import csv
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

raw_train_df = pd.read_csv('fake_or_real_news_training.csv')
raw_test_df = pd.read_csv('fake_or_real_news_test.csv')

raw_train_df.head()


,ID,title,text,label,X1,X2
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,NaN,NaN
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,NaN,NaN
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,NaN,NaN
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,NaN,NaN
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,NaN,NaN


In [2]:
raw_train_df.shape

(3999, 6)

In [3]:
raw_test_df.shape

(2321, 3)

In [4]:
raw_test_df.head()

,ID,title,text
0,10498,September New Homes Sales Rise——-Back To 1992 ...,September New Homes Sales Rise Back To 1992 Le...
1,2439,Why The Obamacare Doomsday Cult Can't Admit It...,But when Congress debated and passed the Patie...
2,864,"Sanders, Cruz resist pressure after NY losses,...",The Bernie Sanders and Ted Cruz campaigns vowe...
3,4128,Surviving escaped prisoner likely fatigued and...,Police searching for the second of two escaped...
4,662,Clinton and Sanders neck and neck in Californi...,No matter who wins California's 475 delegates ...


In [32]:
prepared_train_df = raw_train_df.copy()
prepared_test_df = raw_test_df.copy()

In [6]:
prepared_test_df['label'] = None
prepared_test_df.head()

,ID,title,text,label
0,10498,September New Homes Sales Rise——-Back To 1992 ...,September New Homes Sales Rise Back To 1992 Le...,None
1,2439,Why The Obamacare Doomsday Cult Can't Admit It...,But when Congress debated and passed the Patie...,None
2,864,"Sanders, Cruz resist pressure after NY losses,...",The Bernie Sanders and Ted Cruz campaigns vowe...,None
3,4128,Surviving escaped prisoner likely fatigued and...,Police searching for the second of two escaped...,None
4,662,Clinton and Sanders neck and neck in Californi...,No matter who wins California's 475 delegates ...,None


In [7]:
prepared_train_df['combined'] = prepared_train_df['title'] + prepared_train_df['text']
prepared_test_df['combined'] = prepared_test_df['title'] + prepared_test_df['text']
prepared_train_df.head()

,ID,title,text,label,X1,X2,combined
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,NaN,NaN,"You Can Smell Hillary’s FearDaniel Greenfield,..."
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,NaN,NaN,Watch The Exact Moment Paul Ryan Committed Pol...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,NaN,NaN,Kerry to go to Paris in gesture of sympathyU.S...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,NaN,NaN,Bernie supporters on Twitter erupt in anger ag...
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,NaN,NaN,The Battle of New York: Why This Primary Matte...


# Train/Test Split of Data

In [37]:
def score_model(df, model, vectorizer, dependent_var, size, seed, return_model=True):
    df_new = df.copy()
    
    X = df_new.loc[:, 'text']
    y = df_new.loc[:, dependent_var]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=size, random_state=seed)

    pipeline = make_pipeline(vectorizer, model).fit(X=X_train, y=y_train)

    pred = pipeline.predict(X_test)

    if return_model == False:
        return(pipeline.score(X_test, y_test), y_test, pred)
    else:
        return(pipeline.score(X_test, y_test), y_test, pred, pipeline)


def cv_evaluate(df, model, vectorizer, target_var, seed, cv, metric='accuracy'):
    df_new = df.copy()
    kfolds = KFold(n_splits=cv, shuffle=True, random_state=seed)

    X = df_new.loc[:, 'text']
    y = df_new.loc[:, target_var]
    benchmark_model = make_pipeline(vectorizer, model).fit(X=X, y=y.values.ravel())
    scores = cross_val_score(benchmark_model, X, y.values.ravel(),
                             scoring=metric, cv=kfolds)
    return scores[scores >= 0.0]


def vectorizer(df, dependent_var, seed, transformer, split=True):
    if split == True:
        X = df.loc[:, 'text']
        y = df.loc[:, dependent_var]
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=size, random_state=seed)

        train = transformer.fit_transform(X_train)
        test = transformer.transform(X_test)
        return(train, test, transformer)
    else:
        count = transformer.fit_transform(df)
        return(count, transformer)


# Plots heatmap of confusion matrix
def confusion_heat_map(test_set, prediction_set, class_names):
    cm = confusion_matrix(test_set, prediction_set, class_names)
    fig, ax = plt.subplots()

    # create heatmap
    g = sns.heatmap(cm, annot=True, cmap="YlGnBu", fmt='g',
                    xticklabels=True, yticklabels=True)
    ax.xaxis.set_label_position("top")
    ax.set_xticklabels(class_names)
    ax.set_yticklabels(class_names)
    plt.title('Confusion matrix', y=1.1)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


def run_full_iteration(df, model, vectorizer, target_var, size, seed, cv=5):
    score, y_test, pred, model = score_model(df, model, vectorizer, target_var, size, seed)
    print("Accuracy: %0.3f" % score)

    accuracy = cv_evaluate(df, model, vectorizer, target_var, seed, cv)
    print('Mean Accuracy after CV: {:.3f} +/- {:.03f}'.format(np.mean(accuracy), np.std(accuracy)))
    print('Best Accuracy after CV: {:.3f}'.format(max(accuracy)))

    confusion_heat_map(y_test, pred, ['FAKE', 'REAL'])


In [42]:
# Set test set size, seed and target metric for the rest of the pipeline
size = 0.20
seed = 12345
target_var = 'label'

models = []
models.append(('MNB', MultinomialNB()))
models.append(('LR', LogisticRegression(solver='lbfgs',multi_class='auto')))
models.append(('KNN', KNeighborsClassifier()))
models.append(('RFC', RandomForestClassifier()))
models.append(('SVM', SVC()))
models.append(('GB', GradientBoostingClassifier()))

vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)

# evaluate each model in turn
results = []
names = []
for name, model in models:
    cv_results = cv_evaluate(prepared_train_df, model, vectorizer, target_var, seed, cv=5)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)


MNB: 0.809956 (0.009593)
LR: 0.907728 (0.005645)
KNN: 0.546883 (0.009206)


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RFC: 0.822955 (0.007376)


In [27]:
model
run_full_iteration(prepared_train_df, model, vectorizer, target_var, size, seed)

TypeError: All intermediate steps should be transformers and implement fit and transform. '<function vectorizer at 0x1a18885ea0>' (type <class 'function'>) doesn't